In [3]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

## Absolute MMR Assignment Problem

Basado en Jordi Pereira, Igor Averbakh, Exact and heuristic algorithms for the interval data robust assignment problem, In Computers & Operations Research, Volume 38, Issue 8, 2011, Pages 1153-1163, ISSN 0305-0548, https://doi.org/10.1016/j.cor.2010.11.009.

Utilizaremos el siguiente modelo

$\min \sum_{(u,v)\in E} c_{uv}^+ x_{uv} - \theta$


s.t.

$\theta \leq \sum_{(u,v)\in E} c_{uv}^- y_{uv} + \sum_{(u,v)\in E} y_{uv} (c_{uv}^+ - c_{uv}^-) x_{uv}\qquad \forall Y\in A$

$\sum_{u\in U} x_{uv} = 1, \qquad v\in V$;

$\sum_{v\in V} x_{uv} = 1, \qquad u\in U$;

$x_{uv}\in \{0,1\}, \qquad u\in U, v\in V$.

Nótese que la restricción que calcula $\theta$ hace uso de diversas propiedades del problema (sabemos que el escenario que maximiza la diferencia entre dos soluciones es un escenario extremo de características concretas y que una de esas soluciones define la peor alternativa y por tanto el worst-case regret de una solución $X$.

En realidad es más conveniente implementar como:

$\min \theta$


s.t.

$\theta \geq \sum_{(u,v)\in E} c_{uv}^s x_{uv} - c_{uv}^s x_{uv}^{\ast}\forall s\in S$

y específicamente definir $s$ para una solución $x$ como su caso extremo. Esto es, $c_{uv}^s= c_{uv}^+$ si $x_{uv}=1$ y $c_{uv}^s= c_{uv}^-$ en el resto de los casos.

In [4]:
function doInstanciaAleatoria(n::Int64)
    cplus = rand(1:1000000, n*n)
    cminus=map(x -> rand(1:x), cplus)
    return reshape(cminus,n,n),reshape(cplus,n,n)
end

doInstanciaAleatoria (generic function with 1 method)

In [5]:
srand(0)
cminus,cplus=doInstanciaAleatoria(15)

(
[46212 227032 … 594252 384268; 282754 404683 … 80822 529592; … ; 256077 100788 … 695986 130348; 84569 130215 … 42096 11657],

[50241 229958 … 734015 665675; 498707 524136 … 104588 898982; … ; 375631 140089 … 737225 519511; 152652 153509 … 55125 35154])

In [6]:
function solveAP(c,n::Int64,solver)
    submodel = Model(solver = solver)
    @variable(submodel, x[1:n,1:n],Bin)
    @objective(submodel,Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:n) )
    
    @constraint(submodel, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(submodel, destination[j = 1:n], sum(x[1:n,j]) == 1)
    status = solve(submodel)
    #print("solucion: ",status,"\n\n\n")
    valor=getobjectivevalue(submodel)
    return valor
end

solveAP (generic function with 1 method)

In [5]:
function solveRAP(cminus, cplus, n::Int64, solver=CplexSolver())
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    
    
    function lazyGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    return
                end
            end
        end
        # tenemos una solucion entera, buscamos los costos
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        #print("*")
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido

    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

solveRAP (generic function with 2 methods)

In [ ]:
solveRAP(cminus,cplus,15)

## Compliquemos ligeramente el ejemplo anterior

Tenemos dos puntos que podríamos mejorar:

(1) No estamos generando cortes para las soluciones fraccionales

(2) Vamos a usar la heuristic callback para generar posibles soluciones. Debido a la particularidad de la lazy cut, ésta será la encargada de insertar los cortes

La forma de implementarla va a ser un tanto "rara" y se debe a particularidades de este problema. 

Adicionalmente añadimos una "information callback"

In [7]:
#Instead of x += y, use append!(x,y) to modify x in place. If y is a single variable, you may also use push!(x, coef, y) in place of x += coef*y.                


function solveRAPwithHeuristic(cminus, cplus, n::Int64, solver=CplexSolver(CPX_PARAM_SCRIND=0))
    m = Model(solver = solver)
    
    @variable(m, x[1:n,1:n], Bin)
    @variable(m,θ)

    @objective(m, Min, θ )
    
    @constraint(m, origin[i = 1:n], sum(x[i,1:n]) == 1)
    @constraint(m, destination[j = 1:n], sum(x[1:n,j]) == 1)
    
    cMidPoint=zeros(Float64,n,n)
    for i in 1:n
        for j in 1:n
            cMidPoint[i,j]=cminus[i,j]+cplus[i,j]
        end
    end
    optimalCost = solveAP(cMidPoint,n,CplexSolver(CPX_PARAM_SCRIND=0))
    rhs = - optimalCost 
    for i in 1:n
        for j in 1:n
            rhs += cMidPoint[i,j]*x[i,j]
        end
    end
    @constraint(m,θ >=rhs)        
    #print(m)

    tt = 0.0
    called = 0.0
    separationtime=0.0
    separated=0
    pending = JuMP.GenericAffExpr[]
    function heuristic(m)
        EPSILON=0.00001
        x_val = getvalue(x)
        xTest=zeros(Int64,n)
        yUsed=zeros(Int64,n)
        bestVal=0
        for i in 1:n
            val=(-1)
            bestVal=0
            for j in 1:n
                if yUsed[j]==0 && val<x_val[i,j]
                    val=x_val[i,j]
                    bestVal=j
                end
            end
            xTest[i]=bestVal
            yUsed[bestVal]=1
        end
        
        cost=deepcopy(cminus)        
        for i in 1:n
            x_val[i,:]=0
            x_val[i,xTest[i]]=1
            cost[i,xTest[i]]=cplus[i,xTest[i]]
            #for j in 1:n
            #    if x_val[i,j]>EPSILON
            #        cost[i,j]=cplus[i,j]
            #    end
            #end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        evaluacionCost = - optimalCost
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
                if x_val[i,j]>EPSILON
                    evaluacionCost += cost[i,j]
                end
            end
        end
        #println("\nIn: ",MathProgBase.cbgetobj(m)," vs ",evaluacionCost)
        if(MathProgBase.cbgetobj(m)>evaluacionCost)
            push!(pending,rhs)
            setsolutionvalue(m,θ,evaluacionCost)
            for i in 1:n
                for j in 1:n
                    if x_val[i,j]>EPSILON
                        setsolutionvalue(m,x[i,j],1)
                    else
                        setsolutionvalue(m,x[i,j],0)
                    end
                end
            end
            addsolution(m)
        end
    end
    function lazyGenerator(m)
        #print("\n\n entro en generador de cortes")
        EPSILON=0.00001
        tt = time()
        called += 1
        x_val = getvalue(x)
                
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON && x_val[i,j]<(1.0-EPSILON)
                    return
                end
            end
        end
        # tenemos una solucion entera, buscamos los costos
        cost=deepcopy(cminus)
        for i in 1:n
            for j in 1:n
                if x_val[i,j]>EPSILON
                    cost[i,j]=cplus[i,j]
                end
            end
        end
        optimalCost = solveAP(cost,n,CplexSolver(CPX_PARAM_SCRIND=0))
        #println("\t\t",optimalCost,"\n")
       
        rhs = - optimalCost 
        for i in 1:n
            for j in 1:n
                rhs += cost[i,j]*x[i,j]
            end
        end
        #println(rhs)
        @lazyconstraint(m, θ >= rhs )
        separated += 1
        separationtime += time()-tt
    end
    function cutGenerator(m)
        while !isempty(pending)
            rhs = pop!(pending)
            @usercut(m,θ>=rhs)
        end
    end
    function infocallback(m)
        println("exploredNodes: ",MathProgBase.cbgetexplorednodes(m)," obj: ",MathProgBase.cbgetobj(m), " bestBound: ",MathProgBase.cbgetbestbound(m))        
    end
    #:MIPNode if at node in branch-and-cut tree, 
    #:MIPSol at an integer-feasible solution,
    #:Intermediate otherwise.
    addinfocallback(m, infocallback, when = :MIPNode)
    addlazycallback(m, lazyGenerator) #es necesario un corte lazy ya que le indica al programa que no la he incluido
    addcutcallback(m,cutGenerator)
    addheuristiccallback(m,heuristic)
    status = solve(m)
    println(status)
    println("maxRegret: ",getobjectivevalue(m))
end

solveRAPwithHeuristic (generic function with 2 methods)

In [8]:
solveRAPwithHeuristic(cminus,cplus,15)

exploredNodes: 0 obj: 1.0e75 bestBound: 364275.94887667143
exploredNodes: 0 obj: 1.694855e6 bestBound: 523675.53448333294
exploredNodes: 0 obj: 1.694855e6 bestBound: 542305.8375817484
exploredNodes: 0 obj: 1.694855e6 bestBound: 542305.8375817484
exploredNodes: 0 obj: 1.200952e6 bestBound: 568842.8606790141
exploredNodes: 1 obj: 1.200952e6 bestBound: 643572.8953932837
exploredNodes: 1 obj: 1.200952e6 bestBound: 643572.8953932837
exploredNodes: 2 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 2 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 3 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 3 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 4 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 4 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 5 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 5 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNodes: 6 obj: 1.200952e6 bestBound: 678539.6989798491
exploredNo

exploredNodes: 75 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 76 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 76 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 77 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 77 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 78 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 79 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 79 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 80 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 80 obj: 1.200952e6 bestBound: 710611.9409099177
exploredNodes: 84 obj: 1.200952e6 bestBound: 719613.7836601563
exploredNodes: 84 obj: 1.200952e6 bestBound: 719613.7836601563
exploredNodes: 85 obj: 1.200952e6 bestBound: 723669.4166532855
exploredNodes: 85 obj: 1.200952e6 bestBound: 723669.4166532855
exploredNodes: 86 obj: 1.200952e6 bestBound: 723669.4166532855
exploredNodes: 86 obj: 1.200952e6 bestBound: 723669.416

exploredNodes: 121 obj: 1.640952e6 bestBound: 737423.838441673
exploredNodes: 121 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 122 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 123 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 123 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 125 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 125 obj: 1.200952e6 bestBound: 737423.838441673
exploredNodes: 126 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 126 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 127 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 127 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 128 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 128 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 129 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 129 obj: 1.200952e6 bestBound: 738799.1832417403
exploredNodes: 130 obj: 1.200952e6 bestBound: 7

exploredNodes: 194 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 194 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 195 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 195 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 196 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 196 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 197 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 197 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 198 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 199 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 199 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 202 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 202 obj: 1.200952e6 bestBound: 762438.2632210163
exploredNodes: 203 obj: 1.200952e6 bestBound: 768588.9812035543
exploredNodes: 203 obj: 1.200952e6 bestBound: 768588.9812035543
exploredNodes: 204 obj: 1.200952e6 bestB

exploredNodes: 278 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 278 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 279 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 279 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 280 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 280 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 281 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 281 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 282 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 282 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 285 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 285 obj: 1.200952e6 bestBound: 797709.2598165072
exploredNodes: 286 obj: 1.200952e6 bestBound: 799122.4608582177
exploredNodes: 286 obj: 1.200952e6 bestBound: 799122.4608582177
exploredNodes: 287 obj: 1.200952e6 bestBound: 799122.4608582177
exploredNodes: 287 obj: 1.200952e6 bestB

...

exploredNodes: 2109 obj: 1.200952e6 bestBound: 1.0869002925212164e6
exploredNodes: 2109 obj: 1.200952e6 bestBound: 1.0869002925212164e6
exploredNodes: 2111 obj: 1.200952e6 bestBound: 1.0877014717636819e6
exploredNodes: 2111 obj: 1.200952e6 bestBound: 1.0877014717636819e6
exploredNodes: 2114 obj: 1.200952e6 bestBound: 1.0878016737939883e6
exploredNodes: 2114 obj: 1.200952e6 bestBound: 1.0878016737939883e6
exploredNodes: 2115 obj: 1.200952e6 bestBound: 1.0878853509223964e6
exploredNodes: 2115 obj: 1.200952e6 bestBound: 1.0878853509223964e6
exploredNodes: 2120 obj: 1.200952e6 bestBound: 1.0887169692995765e6
exploredNodes: 2120 obj: 1.200952e6 bestBound: 1.0887169692995765e6
exploredNodes: 2122 obj: 1.200952e6 bestBound: 1.0891650287155185e6
exploredNodes: 2122 obj: 1.200952e6 bestBound: 1.0891650287155185e6
exploredNodes: 2123 obj: 1.200952e6 bestBound: 1.0893959735808482e6
exploredNodes: 2123 obj: 1.200952e6 bestBound: 1.0893959735808482e6
exploredNodes: 2124 obj: 1.200952e6 bestBound: 1

exploredNodes: 2240 obj: 1.200952e6 bestBound: 1.10036557765632e6
exploredNodes: 2240 obj: 1.200952e6 bestBound: 1.10036557765632e6
exploredNodes: 2241 obj: 1.200952e6 bestBound: 1.10036557765632e6
exploredNodes: 2241 obj: 1.200952e6 bestBound: 1.10036557765632e6
exploredNodes: 2242 obj: 1.200952e6 bestBound: 1.10036557765632e6
exploredNodes: 2242 obj: 1.200952e6 bestBound: 1.10036557765632e6
exploredNodes: 2247 obj: 1.200952e6 bestBound: 1.100607624831792e6
exploredNodes: 2247 obj: 1.200952e6 bestBound: 1.100607624831792e6
exploredNodes: 2248 obj: 1.200952e6 bestBound: 1.1007376232990634e6
exploredNodes: 2248 obj: 1.200952e6 bestBound: 1.1007376232990634e6
exploredNodes: 2249 obj: 1.200952e6 bestBound: 1.1007376232990634e6
exploredNodes: 2249 obj: 1.200952e6 bestBound: 1.1007376232990634e6
exploredNodes: 2250 obj: 1.200952e6 bestBound: 1.1007376232990634e6
exploredNodes: 2250 obj: 1.200952e6 bestBound: 1.1007376232990634e6
exploredNodes: 2254 obj: 1.200952e6 bestBound: 1.1007672980659

exploredNodes: 2362 obj: 1.200952e6 bestBound: 1.1102862652689125e6
exploredNodes: 2362 obj: 1.200952e6 bestBound: 1.1102862652689125e6
exploredNodes: 2364 obj: 1.200952e6 bestBound: 1.1105150873233045e6
exploredNodes: 2364 obj: 1.200952e6 bestBound: 1.1105150873233045e6
exploredNodes: 2365 obj: 1.200952e6 bestBound: 1.1107146965575442e6
exploredNodes: 2365 obj: 1.200952e6 bestBound: 1.1107146965575442e6
exploredNodes: 2368 obj: 1.200952e6 bestBound: 1.1115860660369701e6
exploredNodes: 2368 obj: 1.200952e6 bestBound: 1.1115860660369701e6
exploredNodes: 2369 obj: 1.200952e6 bestBound: 1.1118357596123256e6
exploredNodes: 2369 obj: 1.200952e6 bestBound: 1.1118357596123256e6
exploredNodes: 2374 obj: 1.200952e6 bestBound: 1.11220623137221e6
exploredNodes: 2374 obj: 1.200952e6 bestBound: 1.11220623137221e6
exploredNodes: 2376 obj: 1.200952e6 bestBound: 1.112220591160684e6
exploredNodes: 2376 obj: 1.200952e6 bestBound: 1.112220591160684e6
exploredNodes: 2377 obj: 1.200952e6 bestBound: 1.11222

exploredNodes: 2527 obj: 1.200952e6 bestBound: 1.1257495254391676e6
exploredNodes: 2528 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2528 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2529 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2529 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2530 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2530 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2533 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2533 obj: 1.200952e6 bestBound: 1.1259854464592221e6
exploredNodes: 2534 obj: 1.200952e6 bestBound: 1.1260557537482355e6
exploredNodes: 2534 obj: 1.200952e6 bestBound: 1.1260557537482355e6
exploredNodes: 2535 obj: 1.200952e6 bestBound: 1.1260557537482355e6
exploredNodes: 2535 obj: 1.200952e6 bestBound: 1.1260557537482355e6
exploredNodes: 2539 obj: 1.200952e6 bestBound: 1.1262955312892979e6
exploredNodes: 2539 obj: 1.200952e6 bestBound: 1

exploredNodes: 2742 obj: 1.200952e6 bestBound: 1.1475994614341157e6
exploredNodes: 2742 obj: 1.200952e6 bestBound: 1.1475994614341157e6
exploredNodes: 2748 obj: 1.200952e6 bestBound: 1.1484481874902025e6
exploredNodes: 2748 obj: 1.200952e6 bestBound: 1.1484481874902025e6
exploredNodes: 2755 obj: 1.200952e6 bestBound: 1.1490448485370022e6
exploredNodes: 2755 obj: 1.200952e6 bestBound: 1.1490448485370022e6
exploredNodes: 2757 obj: 1.200952e6 bestBound: 1.1492805812414663e6
exploredNodes: 2757 obj: 1.200952e6 bestBound: 1.1492805812414663e6
exploredNodes: 2758 obj: 1.200952e6 bestBound: 1.1493353322637738e6
exploredNodes: 2758 obj: 1.200952e6 bestBound: 1.1493353322637738e6
exploredNodes: 2765 obj: 1.200952e6 bestBound: 1.150101859300306e6
exploredNodes: 2765 obj: 1.200952e6 bestBound: 1.150101859300306e6
exploredNodes: 2770 obj: 1.200952e6 bestBound: 1.1518332348118557e6
exploredNodes: 2770 obj: 1.200952e6 bestBound: 1.1518332348118557e6
exploredNodes: 2771 obj: 1.200952e6 bestBound: 1.1